In [27]:
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-27-3ffd169b915e>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [28]:
db_conn = db.get_db_conn()

## Receving stats by game

In [29]:
query = """
    SELECT
        year,
        season_type,
        posteam AS team,
        receiver,
        week,
        defteam as opp,
        SUM(complete_pass) AS receptions,
        SUM(pass_attempt) AS targets,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
    GROUP BY
        year, week, defteam, posteam, receiver, season_type
    ORDER BY 
        epa DESC
"""
df_game = pd.read_sql(query, db_conn)
print(df_game.shape)
df_game.head(10)

(4754, 15)


,year,season_type,team,receiver,week,opp,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,REG,KC,T.Hill,12,TB,13.0,15.0,269.0,226.0,188.0,3.0,0.0,0.0,22.534387
1,2020,REG,GB,D.Adams,7,HOU,13.0,16.0,196.0,134.0,94.0,2.0,0.0,0.0,17.767565
2,2020,REG,LV,D.Waller,13,NYJ,13.0,17.0,200.0,134.0,92.0,2.0,0.0,0.0,16.420833
3,2020,REG,GB,D.Adams,9,SF,10.0,12.0,173.0,157.0,110.0,1.0,0.0,0.0,15.612783
4,2020,REG,ATL,J.Jones,6,MIN,8.0,10.0,137.0,79.0,70.0,2.0,0.0,0.0,15.549295
5,2020,REG,CIN,T.Boyd,2,CLE,7.0,8.0,72.0,45.0,45.0,1.0,0.0,0.0,15.152323
6,2020,REG,TEN,A.Brown,17,HOU,10.0,11.0,151.0,128.0,117.0,1.0,0.0,0.0,15.026294
7,2020,REG,SF,G.Kittle,4,PHI,15.0,15.0,183.0,109.0,109.0,1.0,0.0,0.0,14.646846
8,2020,REG,NE,D.Byrd,11,HOU,6.0,7.0,132.0,125.0,106.0,1.0,0.0,0.0,14.584462
9,2020,REG,TEN,C.Davis,13,CLE,11.0,12.0,182.0,137.0,132.0,1.0,0.0,0.0,14.222169


In [11]:
# Calvin Ridley is off by 7

query = """
    SELECT
        year,
        season_type,
        posteam AS team,
        receiver,
        position,
        COUNT(DISTINCT game_id) as games,
        SUM(complete_pass) AS receptions,
        SUM(pass_attempt) AS targets,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
    GROUP BY
        year, posteam, position, receiver, season_type  
    ORDER BY
        yards DESC
"""
df_year = pd.read_sql(query, db_conn)
print(df_year.shape)
df_year.head(10)

(696, 15)


,year,season_type,team,receiver,position,games,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,REG,BUF,S.Diggs,WR,16,127.0,166.0,1535.0,1674.0,1071.0,8.0,2.0,0.0,84.674506
1,2020,REG,KC,T.Kelce,TE,15,105.0,145.0,1416.0,1220.0,833.0,11.0,3.0,3.0,74.666660
2,2020,REG,ARI,D.Hopkins,WR,16,115.0,160.0,1407.0,1423.0,879.0,6.0,2.0,3.0,53.923376
3,2020,REG,MIN,J.Jefferson,WR,16,88.0,125.0,1400.0,1421.0,943.0,7.0,4.0,1.0,65.025975
4,2020,REG,ATL,C.Ridley,WR,15,91.0,144.0,1381.0,2059.0,1106.0,9.0,5.0,1.0,51.099656
5,2020,REG,GB,D.Adams,WR,14,115.0,149.0,1374.0,1312.0,777.0,18.0,3.0,1.0,77.433598
6,2020,REG,SEA,D.Metcalf,WR,16,83.0,129.0,1303.0,1768.0,940.0,10.0,4.0,1.0,59.387525
7,2020,REG,KC,T.Hill,WR,15,87.0,135.0,1276.0,1713.0,816.0,15.0,1.0,0.0,75.457701
8,2020,REG,CHI,A.Robinson,WR,16,102.0,151.0,1250.0,1454.0,931.0,6.0,5.0,0.0,35.249572
9,2020,REG,LV,D.Waller,TE,16,107.0,146.0,1196.0,1135.0,624.0,9.0,3.0,2.0,55.804421


In [31]:
df_year.to_csv("rec_game.csv", index=False)

## Debugging - calvin ridley is over by 7 yards

In [33]:
## The game vs detroit is the culprit - week 7
df = df_game[df_game['receiver'] == 'C.Ridley']
df.sort_values('week').head(20)

,year,season_type,team,receiver,week,opp,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
65,2020,REG,ATL,C.Ridley,1,SEA,9.0,12.0,130.0,141.0,109.0,2.0,0.0,0.0,9.588496
92,2020,REG,ATL,C.Ridley,2,DAL,7.0,10.0,109.0,170.0,95.0,2.0,0.0,0.0,9.000807
4302,2020,REG,ATL,C.Ridley,3,CHI,5.0,13.0,110.0,249.0,92.0,0.0,1.0,0.0,-2.628966
4680,2020,REG,ATL,C.Ridley,4,GB,0.0,5.0,0.0,98.0,0.0,0.0,0.0,0.0,-6.860108
26,2020,REG,ATL,C.Ridley,5,CAR,8.0,10.0,136.0,132.0,111.0,0.0,0.0,0.0,11.408379
496,2020,REG,ATL,C.Ridley,6,MIN,6.0,7.0,61.0,56.0,42.0,1.0,0.0,0.0,4.644250
330,2020,REG,ATL,C.Ridley,7,DET,6.0,8.0,76.0,102.0,75.0,1.0,0.0,0.0,5.811320
1715,2020,REG,ATL,C.Ridley,8,CAR,3.0,4.0,42.0,54.0,31.0,0.0,1.0,0.0,1.199328
3655,2020,REG,ATL,C.Ridley,11,NO,5.0,9.0,90.0,161.0,79.0,0.0,1.0,0.0,-1.097890
168,2020,REG,ATL,C.Ridley,12,LV,6.0,9.0,50.0,78.0,32.0,1.0,0.0,0.0,7.563623


In [34]:
df.to_csv("rec_game.csv", index=False)

### Matt ryan has the proper yards/attempts - ridley has the one extra target?

In [23]:
## id 932 is the culprit - the pass actually whent to hayden hurts
query = """
    SELECT
        play_id,
        "desc",
        year,
        season_type,
        posteam AS team,
        receiver,
        week,
        defteam as opp,
        SUM(complete_pass) AS receptions,
        SUM(pass_attempt) AS targets,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
        AND receiver = 'C.Ridley'
        AND week = 7
    GROUP BY
        play_id, "desc", year, week, defteam, posteam, receiver, season_type
    ORDER BY 
        epa DESC
"""
df = pd.read_sql(query, db_conn)
print(df.shape)
df.head(10)

(8, 17)


,play_id,desc,year,season_type,team,receiver,week,opp,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,1714,(5:32) (Shotgun) 2-M.Ryan pass deep left to 18-C.Ridley to ATL 31 for 27 yards.,2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,27.0,26.0,26.0,0.0,0.0,0.0,2.756491
1,2023,"(:34) (Shotgun) 2-M.Ryan pass short middle to 18-C.Ridley for 4 yards, TOUCHDOWN.",2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,4.0,4.0,4.0,1.0,0.0,0.0,2.601103
2,1160,(13:48) (Shotgun) 2-M.Ryan pass deep right to 18-C.Ridley ran ob at DET 22 for 19 yards.,2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,19.0,19.0,19.0,0.0,0.0,0.0,1.312245
3,3552,(7:32) (Shotgun) 2-M.Ryan pass short left to 18-C.Ridley to ATL 48 for 12 yards (24-A.Oruwariye).,2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,12.0,12.0,12.0,0.0,0.0,0.0,0.559128
4,932,"(:31) (Shotgun) 2-M.Ryan pass incomplete short right to 18-C.Ridley. Atlanta challenged the incomplete pass ruling, and the play was REVERSED. (Shotgun) 2-M.Ryan pass short middle to 81-H.Hurst to ATL 42 for 7 yards (71-D.Shelton). Pass initially thrown to 18-Ridley to the short right. 81-Hurst caught the deflection.",2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,7.0,7.0,7.0,0.0,0.0,0.0,0.398989
5,3191,(12:39) (Shotgun) 2-M.Ryan pass short right to 18-C.Ridley ran ob at DET 13 for 7 yards [95-R.Okwara].,2020,REG,ATL,C.Ridley,7,DET,1.0,1.0,7.0,7.0,7.0,0.0,0.0,0.0,-0.402150
6,954,(:12) 2-M.Ryan pass incomplete deep left to 18-C.Ridley.,2020,REG,ATL,C.Ridley,7,DET,0.0,1.0,0.0,23.0,0.0,0.0,0.0,0.0,-0.500036
7,2001,(:42) 2-M.Ryan pass incomplete short left to 18-C.Ridley.,2020,REG,ATL,C.Ridley,7,DET,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,-0.914451


### Ok so ridley got a pass that was reversed and given to hurst...

In [24]:
## The game vs detroit is the culprit - week 7
df = df_game[df_game['receiver'] == 'H.Hurst']
df.sort_values('week').head(20)

,year,season_type,team,receiver,week,opp,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
2784,2020,REG,ATL,H.Hurst,1,SEA,3.0,5.0,38.0,38.0,13.0,0.0,0.0,0.0,-0.268837
1051,2020,REG,ATL,H.Hurst,2,DAL,5.0,8.0,72.0,60.0,26.0,1.0,0.0,0.0,2.582989
3490,2020,REG,ATL,H.Hurst,3,CHI,1.0,3.0,1.0,11.0,1.0,1.0,0.0,0.0,-0.878364
1500,2020,REG,ATL,H.Hurst,4,GB,4.0,6.0,51.0,89.0,36.0,0.0,0.0,0.0,1.540615
4524,2020,REG,ATL,H.Hurst,5,CAR,2.0,6.0,8.0,55.0,7.0,0.0,0.0,0.0,-4.089225
106,2020,REG,ATL,H.Hurst,6,MIN,4.0,4.0,57.0,21.0,21.0,1.0,0.0,0.0,8.707816
141,2020,REG,ATL,H.Hurst,7,DET,5.0,6.0,61.0,40.0,34.0,0.0,0.0,0.0,8.105338
1379,2020,REG,ATL,H.Hurst,8,CAR,5.0,7.0,54.0,40.0,33.0,0.0,0.0,0.0,1.831763
1546,2020,REG,ATL,H.Hurst,9,DEN,7.0,8.0,62.0,40.0,22.0,0.0,0.0,0.0,1.468144
4330,2020,REG,ATL,H.Hurst,11,NO,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.740990


### Sure enoough hurst is missing the 1 target for 7 yards

In [26]:
query = """
    SELECT
        *
    FROM
        play_by_play_enriched
    WHERE
        posteam = 'ATL' and play_id = 932 and week = 7
"""
df = pd.read_sql(query, db_conn)
print(df.shape)
df.head(10)

(1, 343)


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,gsis_id,position
0,932,2020_07_DET_ATL,2020102500,ATL,DET,REG,7,ATL,home,DET,ATL,65.0,2020-10-25,31.0,931.0,2731.0,Half1,0,7.0,0,1,2.0,0,00:31,ATL 35,2,89.0,"(:31) (Shotgun) 2-M.Ryan pass incomplete short right to 18-C.Ridley. Atlanta challenged the incomplete pass ruling, and the play was REVERSED. (Shotgun) 2-M.Ryan pass short middle to 81-H.Hurst to ATL 42 for 7 yards (71-D.Shelton). Pass initially thrown to 18-Ridley to the short right. 81-Hurst caught the deflection.",pass,7.0,1,0,1.0,0,0,0,short,right,7.0,0.0,None,None,None,None,None,None,3,3,0.0,None,None,3.0,3.0,0,7,0.0,7.0,-7.0,0.0,7.0,-7.0,0.022926,0.118418,0.002195,0.193064,0.224666,0.003298,0.435433,0.0,0.0,2.017531,0.398989,-7.750275,7.750275,-2.306319,2.306319,-1.686641,1.686641,0.398989,0.0,0.398989,0.0,-4.478143,4.478143,4.573825,-4.573825,-12.152177,12.152177,10.465536,-10.465536,0.34776,0.65224,0.34776,0.65224,-0.004755,0.343004,0.656996,0.342817,0.342817,-0.092201,...,None,None,None,None,None,None,0.0,None,None,None,None,1,reversed,None,0.0,0.0,0.0,0.0,2020,0.75798,24.201971,13,1,First down,932.0,13:00:00,17:40:13,Mercedes-Benz Stadium,"Cloudy Temp: 71° F, Humidity: 79%, Wind: NW 3 mph",10160000-0582-627f-9a10-b59968125ce5,11,0,PAS